<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/faiss_locality_sensitive_hashing_random_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing-random-projection/
2. How LSH Random Projection works in search (+Python): https://www.youtube.com/watch?v=8bOrMqEdfiQ
3. IndexLSH for Fast Similarity Search in Faiss: https://www.youtube.com/watch?v=ZLfdQq_u7Eo


In [ ]:
!apt install libomp-dev
!pip install --upgrade faiss-cpu faiss-gpu


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (1,119 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

![](https://cdn.sanity.io/images/vr8gru94/production/d4a50340e4cf272f36f9c278c3210bf51f3bc7e8-1400x787.png)

局部敏感哈希（LSH）是近似相似性搜索中广泛使用的技术。高效相似性搜索的解决方案是一种有利可图的解决方案——它是数十亿（甚至数万亿）美元公司的核心。

相似性搜索的问题在于规模。许多公司每天处理数百万至数十亿的数据点。给定十亿个数据点，是否可以在每次搜索时对所有数据点进行比较？

此外，许多公司并不执行单一搜索 - Google 每分钟处理超过 380 万次搜索[1]。

数十亿个数据点与高频搜索相结合是有问题的——而且我们没有考虑维度或相似性函数本身。显然，对于较大的数据集来说，对所有数据点进行详尽的搜索是不现实的。

搜索巨大数据集的解决方案？近似搜索。我们不是详尽地比较每一对，而是近似- 将搜索范围仅限于高概率匹配。

## 局部敏感哈希

[局部敏感哈希（LSH）](https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/)是最流行的近似最近邻搜索（ANNS）方法之一。

从本质上讲，它是一个哈希函数，允许我们将相似的项分组到相同的哈希桶中。因此，给定一个难以置信的巨大数据集 - 我们通过哈希函数运行所有项目，将项目分类到存储桶中。

与大多数旨在最小化哈希冲突的哈希函数不同，LSH 算法旨在最大化哈希冲突。

![两个哈希函数，顶部（蓝色）最大限度地减少哈希冲突。 底部（洋红色）最大化散列冲突 - LSH 旨在最大化相似项之间的冲突。](https://cdn.sanity.io/images/vr8gru94/production/f3bc4a56f42d31d08824ab5f9fdfea049a067797-1280x720.png)

两个哈希函数，顶部（蓝色）最大限度地减少哈希冲突。底部（洋红色）最大化散列冲突 - LSH 旨在最大化相似项之间的冲突。

LSH 的结果是相似的[向量](https://www.pinecone.io/learn/vector-embeddings/)产生相同的哈希值并存储在一起。相反，不同的向量将*不会*产生相同的哈希值——被放置在不同的桶中。

### 使用 LSH 搜索

使用 LSH 执行搜索包括三个步骤：
1. 将我们所有的向量索引到它们的散列向量中。
2. 介绍我们的查询向量（搜索词）。它使用相同的 LSH 函数进行哈希处理。
3. 通过汉明距离将我们的散列查询向量与所有其他散列桶进行比较 - 识别最近的。

在非常高的层面上，这就是我们将要介绍的 LSH 方法的流程。不过，我们将在本文中进一步详细解释所有这些。

### 近似的影响

在深入研究 LSH 的细节之前，我们应该注意，将向量分组为较低分辨率的散列向量也意味着我们的搜索并不详尽（例如，比较每个向量），因此我们预计搜索质量较低。

![我们将潜在的巨大密集向量压缩为高度压缩的分组二进制向量。](https://cdn.sanity.io/images/vr8gru94/production/5a5f928e39a01b106bc5d2e54d1ead84477fc8bd-1280x700.png)

我们将潜在的巨大密集向量压缩为高度压缩的分组二进制向量。

但是，接受这种较低搜索质量的原因是搜索速度可能会显著加快。

### 哪种方法？

我们在这里故意不透露细节，因为 LSH 有多个版本 - 每个版本都使用不同的哈希构建和距离/相似性度量。两种最流行的方法是：

- Shingling、MinHashing 和 banded LSH（传统方法）
- 具有点积和汉明距离的随机超平面

[本文将重点介绍随机超平面方法，该方法在各种流行的库（例如Faiss）](https://www.pinecone.io/learn/series/faiss/)中更常用和实现。

------

## 随机超平面

随机超平面（也称为*随机投影*）方法看似简单，但很难找到该方法的详细信息。

让我们通过一个示例来学习 - 我们将在整个示例中使用 Sift1M 数据集，您可以使用[此脚本](https://gist.github.com/jamescalam/a09a16c17b677f2cf9c019114711f3bf)下载该数据集。

现在，给定一个查询向量xq，我们想要从xb数组中识别前k 个最近邻居。

![在这里，我们返回查询向量 xq 的三个最近邻居。](https://cdn.sanity.io/images/vr8gru94/production/8952c4995a2bd53b890dde38401f86b31a44dbc8-1400x787.png)

在这里，我们返回查询向量 xq 的三个最近邻居。

使用随机投影方法，我们将高维向量减少为低维二元向量。一旦我们有了这些二进制向量，我们就可以使用汉明距离来测量它们之间的距离。

让我们更详细地讨论一下。

### 创建超平面

此方法中的超平面用于分割数据点，并为出现在超平面负侧的数据点指定值0 ，为出现在正侧的数据点指定值1 。

![我们将值 1 分配给超平面 +ve 侧的向量，将值 0 分配给超平面 -ve 侧的向量。](https://cdn.sanity.io/images/vr8gru94/production/c5f4a5d538de6fa256c0828b710fdbf545152289-1400x787.png)

我们将值 1 分配给超平面 +ve 侧的向量，将值 0 分配给超平面 -ve 侧的向量。

为了确定我们的数据点位于超平面的哪一侧，我们需要的是平面的法向量——例如，垂直于平面的向量。我们将这个法线向量（与我们的数据点向量一起）输入到点积函数中。

如果两个向量具有相同的方向，则所得的点积为正。如果它们的方向*不同*，则为负值。

![当我们的超平面法向量与另一个向量产生 +ve 点积时，我们可以将该向量视为位于超平面前面。 对于产生 -ve 点积的向量来说，情况正好相反。](https://cdn.sanity.io/images/vr8gru94/production/c626d2de2c70dd46c495c0c09cc5e1d4d212e02c-1280x720.png)

当我们的超平面法向量与另一个向量产生 +ve 点积时，我们可以将该向量视为位于超平面前面。对于产生 -ve 点积的向量来说，情况正好相反。

*在两个向量完全垂直（位于超平面边缘）的不太可能的情况下，点积为 0 - 我们将其与负方向向量分组。*

单个二进制值并不能告诉我们太多关于向量的相似性的信息，但是当我们开始添加*更多*超平面时，编码信息量会迅速增加。

![我们添加更多超平面来增加二进制向量中存储的位置信息量。](https://cdn.sanity.io/images/vr8gru94/production/a7f8e91533957508ab25f70358768ad116ae37b3-1280x720.png)

我们添加更多超平面来增加二进制向量中存储的位置信息量。

通过使用这些超平面将向量投影到低维空间，我们生成新的*散列*向量。

在上图中，我们使用了两个超平面，实际上我们还需要更多——我们使用nbits参数定义的属性。我们稍后将更详细地讨论nbits，但现在，我们将通过设置nbits = 4来使用四个超平面。

现在，让我们在 Python 中创建超平面的法向量。

In [ ]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

import tarfile
# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])


In [ ]:
xq.shape,xb.shape,xq



((1, 128),
 (1000000, 128),
 array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
          18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
         101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
           1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
          40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
         116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
          10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
           0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
         116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
           9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
           2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
          18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32))

In [ ]:
nbits = 4  # number of hyperplanes and binary vals to produce
d = 2  # vector dimensions

In [ ]:
import numpy as np
# create a set of 4 hyperplanes, with 2 dimensions
plane_norms = np.random.rand(nbits, d) - .5
plane_norms

array([[-0.00070864,  0.13422671],
       [-0.19221854, -0.27650181],
       [-0.0100398 ,  0.04139894],
       [-0.11859579, -0.16500783]])

通过np.random.rand我们创建一组0 → 1范围内的随机值。然后我们添加-.5以使数组值以原点*(0, 0)*为中心。可视化这些向量，我们看到：

![定义超平面位置的法向量，均以原点 (0, 0) 为中心。](https://cdn.sanity.io/images/vr8gru94/production/b17a12452dcb7969f90ed67b971e4f42c16337ea-1280x720.png)

定义超平面位置的法向量，均以原点 (0, 0) 为中心。

### 哈希向量

现在让我们添加三个向量 - **a**、**b**和**c** - 并使用四个法线向量及其超平面构建哈希值。

In [ ]:
a = np.asarray([1, 2])
b = np.asarray([2, 1])
c = np.asarray([3, 1])
a,b,c

(array([1, 2]), array([2, 1]), array([3, 1]))

In [ ]:
# calculate the dot product for each of these
a_dot = np.dot(a, plane_norms.T)
b_dot = np.dot(b, plane_norms.T)
c_dot = np.dot(c, plane_norms.T)
a_dot,b_dot,c_dot

(array([ 0.26774479, -0.74522216,  0.07275807, -0.44861145]),
 array([ 0.13280944, -0.66093889,  0.02131934, -0.40219942]),
 array([ 0.1321008 , -0.85315743,  0.01127954, -0.52079521]))

In [ ]:
# we know that a positive dot product == +ve side of hyperplane
# and negative dot product == -ve side of hyperplane
a_dot = a_dot > 0
b_dot = b_dot > 0
c_dot = c_dot > 0
a_dot,b_dot,c_dot

(array([ True, False,  True, False]),
 array([ True, False,  True, False]),
 array([ True, False,  True, False]))

In [ ]:
# convert our boolean arrays to int arrays to make bucketing
# easier (although is okay to use boolean for Hamming distance)
a_dot = a_dot.astype(int)
b_dot = b_dot.astype(int)
c_dot = c_dot.astype(int)
a_dot,b_dot,c_dot

(array([1, 0, 1, 0]), array([1, 0, 1, 0]), array([1, 0, 1, 0]))

再次可视化，我们得到了三个向量**a**、**b**和**c** - 以及四个超平面（垂直于它们各自的法向量）。分别取 +ve 和 -ve 点积值得出：

![0表示向量位于平面后面（-ve 点积），1表示向量位于平面前面（+ve 点积）。 我们将它们组合起来创建我们的二进制向量。](https://cdn.sanity.io/images/vr8gru94/production/f4fbfddbfa7629ea1e7a717777059c6279c4203d-1280x720.png)

0表示向量位于平面后面（-ve 点积），1表示向量位于平面前面（+ve 点积）。我们将它们组合起来创建我们的二进制向量。

这会产生我们的散列向量。现在，LSH 使用这些值来创建存储桶，其中将包含一些对我们的向量的引用（例如它们的 ID）。请注意，我们不会将原始向量存储在存储桶中，这会显着增加 LSH 索引的大小。

[正如我们将在Faiss](https://www.pinecone.io/learn/series/faiss/faiss-tutorial/)等实现中看到的那样- 通常存储我们添加向量的位置/顺序。我们将在示例中使用相同的方法。



In [ ]:
vectors = [a_dot, b_dot, c_dot]
buckets = {}
i = 0

for i in range(len(vectors)):
    # convert from array to string
    hash_str = ''.join(vectors[i].astype(str))
    # create bucket if it doesn't exist
    if hash_str not in buckets.keys():
        buckets[hash_str] = []
    # add vector position to bucket
    buckets[hash_str].append(i)

print(buckets)

{'1010': [0, 1, 2]}


现在我们已经对三个向量进行了存储，让我们考虑一下搜索复杂性的变化。假设我们引入一个哈希为**0111 的**查询向量。

通过这个向量，我们将它与 LSH 索引中的每个存储桶进行比较——在本例中只有两个值**——1000**和**0110**。然后我们使用汉明距离来找到最接近的匹配，这恰好是**0110**。

![汉明距离，前两个向量之间有四个不匹配，导致汉明距离为 4。 接下来的两个仅包含一个不匹配，汉明距离为 1。](https://cdn.sanity.io/images/vr8gru94/production/95721cf0c2c7689aacc6f8fc27e254249b209c19-1280x720.png)

汉明距离，前两个向量之间有四个不匹配，导致汉明距离为 4。接下来的两个仅包含一个不匹配，汉明距离为 1。

*我们采用了线性复杂度函数，这要求我们计算查询向量与所有先前索引向量之间的距离（达到亚线性复杂度），因为我们不再需要计算每个*向量的距离，因为它们'重新分组到桶中。

现在，向量**1**和**2**同时等于**0110**。所以我们不可能找到其中哪一个最接近我们的查询向量。这意味着搜索质量会受到一定程度的损失——然而，这只是执行近似搜索的成本。*我们以质量换取速度。*

------

## 平衡质量与速度

[正如相似性搜索](https://www.pinecone.io/learn/what-is-similarity-search/)中常见的情况一样，良好的 LSH 索引需要平衡搜索质量与搜索速度。

我们在迷你示例中看到，我们的向量不容易区分，因为在总共三个向量中，随机投影已将其中两个向量哈希为同一个二进制向量。

现在想象我们将相同的比率缩放到包含一百万个向量的数据集。我们引入我们的查询向量**xq —对其进行散列并计算该向量 (** **0111** ) 与我们的*两个*存储桶（**1000**和**0110**）之间的汉明距离。

哇，仅用两次距离计算就搜索了一百万个样本数据集？那*很快*。

不幸的是，我们返回了大约 700K 个样本，所有样本都具有相同的二进制向量值**0110**。是的，快，但是*一点也不准确*。

现在，实际上，使用**nbits**值**4**将产生**16 个**存储桶：

In [ ]:
nbits = 4

# this returns number of binary combos for our nbits val
1 << nbits

16

In [ ]:
# we can print every possible bucket given our nbits value like so
for i in range(1 << nbits):
    # get binary representation of integer
    b = bin(i)[2:]
    # pad zeros to start of binary representtion
    b = '0' * (nbits - len(b)) + b
    print(b, end=' | ')

0000 | 0001 | 0010 | 0011 | 0100 | 0101 | 0110 | 0111 | 1000 | 1001 | 1010 | 1011 | 1100 | 1101 | 1110 | 1111 | 

我们坚持使用**1000**和**0110**这两个桶只是为了产生戏剧性的效果。但即使有*16 个*存储桶 — 1M 个向量仅分成 16 个存储桶，仍然会产生非常不精确的存储桶。

实际上，我们使用更多的超平面——更多的超平面意味着更高分辨率的二进制向量——产生更精确的向量表示。

我们通过具有 nbits 值的超平面来控制该分辨率。较高的**nbits**值可通过提高散列向量的分辨率来提高搜索质量。

![增加 nbits 参数会增加用于构建二进制向量表示的超平面的数量。](https://cdn.sanity.io/images/vr8gru94/production/dc8749921bfba92e496c9ea3fd5e8f911e04f04c-1400x727.png)

增加 nbits 参数会增加用于构建二进制向量表示的超平面的数量。

添加更多可能的哈希值组合会增加存储桶的潜在数量，从而增加比较次数，从而增加*搜索时间*。

In [ ]:
for nbits in [2, 4, 8, 16]:
    print(f"nbits: {nbits}, buckets: {1 << nbits}")

nbits: 2, buckets: 4
nbits: 4, buckets: 16
nbits: 8, buckets: 256
nbits: 16, buckets: 65536


值得注意的是，并非*所有*桶都一定会被使用——特别是对于更高的**nbits**值。通过我们的 Faiss 实现，我们将看到128或更大的nbits值是完全有效的，并且仍然比使用[平面索引(flat index)](https://www.pinecone.io/learn/series/faiss/vector-indexes/)更快。

还有[这个笔记本介绍了 LSH With Random Projection在 Python 中的简单实现](https://github.com/pinecone-io/examples/blob/master/learn/search/faiss-ebook/locality-sensitive-hashing-random-projection/random_projection.ipynb)。

## LSH in Faiss

我们之前讨论过 [Faiss](https://www.pinecone.io/learn/series/faiss/faiss-tutorial/)，但让我们简单回顾一下。Faiss（或 Facebook AI 相似性搜索）是一个开源框架，旨在实现相似性搜索。

[Faiss 有许多不同索引](https://www.pinecone.io/learn/series/faiss/vector-indexes/)的超高效实现，我们可以在相似性搜索中使用它们。这个长长的索引列表包括**IndexLSH** — 一个易于使用的实现，它是我们迄今为止所介绍的所有内容的实现。

我们初始化 LSH 索引并添加 Sift1M 数据集**wb**,如下所示：

In [ ]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

import tarfile
# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# data we will search through
wb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

xq.shape,wb.shape,xq

((1, 128),
 (1000000, 128),
 array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
          18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
         101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
           1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
          40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
         116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
          10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
           0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
         116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
           9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
           2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
          18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32))

In [ ]:
import faiss

d = wb.shape[1]
nbits = 4

# initialize the index using our vectors dimensionality (128) and nbits
index = faiss.IndexLSH(d, nbits)
# then add the data
index.add(wb)

一旦我们的索引准备好了，我们就可以开始使用index.search(xq, k)进行搜索——其中xq是我们的一个或多个查询向量，k是我们想要返回的最近匹配的数量。



In [ ]:
xq0 = xq[0].reshape(1, d)
# we use the search method to find the k nearest vectors
D, I = index.search(xq0, k=10)
# the indexes of these vectors are returned to I
I

array([[ 0,  2,  6, 25, 26, 43, 47, 70, 73, 74]])

search方法返回两个数组。我们的k 个最佳匹配的索引位置I （例如， wb的行号） 。以及这些最佳匹配与我们的查询向量xq0之间的距离D。

## 评估性能 (Measuring Performance)
因为我们在I中有这些索引位置，所以我们可以从数组wb中检索原始向量。



In [ ]:
# we can retrieve the original vectors from wb using I
wb[I[0]]

array([[ 0., 16., 35., ..., 25., 23.,  1.],
       [ 0.,  1.,  5., ...,  4., 23., 10.],
       [ 0., 42., 55., ..., 45., 11.,  2.],
       ...,
       [ 5., 12., 31., ...,  2.,  9.,  8.],
       [35., 14., 15., ..., 10., 16., 32.],
       [ 6., 15.,  9., ..., 12., 17., 21.]], dtype=float32)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# and calculate the cosine similarity between these and xq0
cosine_similarity(wb[I[0]], xq0)

array([[0.2670474 ],
       [0.20448917],
       [0.3054034 ],
       [0.69142616],
       [0.7344476 ],
       [0.6995599 ],
       [0.6316513 ],
       [0.25432232],
       [0.30497947],
       [0.34137398]], dtype=float32)

从这些原始向量中，我们可以看到 LSH 索引是否返回相关结果。我们通过测量查询向量xq0和前k个匹配之间的余弦相似度来做到这一点。

该索引中的某些向量应返回 0.8 左右的相似度分数。我们返回的向量相似度分数仅为 0.2 — 为什么我们会看到如此糟糕的性能？

### 诊断性能问题 (Diagnosing Performance Issues)

我们知道nbits值控制索引中潜在存储桶的数量。初始化此索引时，我们设置nbits == 4 — 因此所有向量必须存储在 4 位低分辨率存储桶中。

如果我们尝试将 1M 个向量塞入 16 个哈希桶中，那么每个桶很可能包含 10-100K+ 向量。

因此，当我们对搜索查询进行哈希处理时，它与这 16 个存储桶之一完美匹配 - 但索引无法区分挤入该单个存储桶的大量向量 - 它们都具有相同的哈希向量！

我们可以通过检查距离D来确认这一点：




In [ ]:
D

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

我们为每个项目返回零的完美距离分数- 但为什么呢？好吧，我们知道，对于完美匹配，汉明距离只能为零——这意味着所有这些散列向量必须相同。

如果所有这些向量都返回完美匹配，则它们必须具有相同的哈希值。因此，我们的指数无法区分它们——就我们的 LSH 指数而言，它们都具有相同的位置。

现在，如果我们增加k直到返回非零距离值，我们应该能够推断出使用相同哈希码存储的向量的数量。让我们继续尝试一下。



In [ ]:
k = 100
xq0 = xq[0].reshape(1, d)

while True:
    D, I = index.search(xq0, k=k)
    if D.any() != 0:
        print(k)
        break
    k += 100

172100


In [ ]:
D  # we will see both 0s, and 1s

array([[0., 0., 0., ..., 1., 1., 1.]], dtype=float32)

In [ ]:
D[:,172_039:172_041]  # we see the hash code switch at position 172_039

array([[0., 1.]], dtype=float32)

包含172_039个向量的单个桶。这意味着我们从这 172K 个向量中随机选择前k个值。显然，我们需要减小存储桶的大小。

对于 1M 样本，哪个nbits值可以为我们提供足够的存储桶来实现更稀疏的向量分布？不可能计算出精确的分布，但我们可以取平均值：



In [ ]:
for nbits in [2, 4, 8, 16, 24, 32]:
    buckets = 1 << nbits
    print(f"nbits == {nbits}")
    print(f"{wb.shape[0]} / {buckets} = {wb.shape[0]/buckets}")

nbits == 2
1000000 / 4 = 250000.0
nbits == 4
1000000 / 16 = 62500.0
nbits == 8
1000000 / 256 = 3906.25
nbits == 16
1000000 / 65536 = 15.2587890625
nbits == 24
1000000 / 16777216 = 0.059604644775390625
nbits == 32
1000000 / 4294967296 = 0.00023283064365386963


当nbits值为 16 时，我们仍然在每个桶中获得平均15.25 个向量 - 这看起来比实际情况要好。我们必须考虑到某些桶将明显大于其他桶，因为不同区域将包含比其他区域更多的向量。

实际上，24和32的nbits值可能是我们实现真正有效的存储桶大小的临界点。让我们找到每个值的平均余弦相似度。



In [ ]:
xq0 = xq[0].reshape(1, d)
k = 100

for nbits in [2, 4, 8, 16, 24, 32]:
    index = faiss.IndexLSH(d, nbits)
    index.add(wb)
    D, I = index.search(xq0, k=k)
    cos = cosine_similarity(wb[I[0]], xq0)
    print(np.mean(cos))

0.54244477
0.56082696
0.6372648
0.6676912
0.7132521
0.7051426


看起来我们的估计是正确的——前 100 个向量的整体相似度随着每个 nbits 的增加而突然增加，然后在 nbits == 24点趋于平稳。但是如果我们使用更大的nbits值运行该进程会怎样？


![当我们使用 nbits 提高向量分辨率时，我们的结果将变得更加精确 - 在这里，我们可以看到较大的 nbits 值会导致结果的余弦相似度更高。](https://cdn.sanity.io/images/vr8gru94/production/c6c6b72078dba1cd0093addff8dc1f5318b31448-1280x720.png)

当我们使用 nbits 提高向量分辨率时，我们的结果将变得更加精确 - 在这里，我们可以看到较大的 nbits 值会导致结果的余弦相似度更高。

到这里，结果就很明显了。当我们整理 LSH 桶时，相似度会快速增加，然后相似度会缓慢增加。

后者相似性增加较慢，这要归功于散列向量分辨率的提高。*我们的存储桶已经非常稀疏——我们拥有的潜在*存储桶比向量多得多，因此我们发现性能几乎没有提高。

*但是*，我们正在提高分辨率，*从而提高这些存储桶的精度*，因此我们从这里获得额外的性能。

### 提取二进制向量

在研究上面的索引和向量在存储桶中的分布时，我们推断出存储桶大小的问题。这很有用，因为我们正在使用我们已经了解的 LSH 知识，并且可以看到索引属性的影响。

然而，我们可以采取更直接的方法。Faiss 允许我们通过提取**wb**的二进制向量表示来*间接*查看我们的存储桶。

让我们恢复到**nbits**值**4**，看看 LSH 索引中存储了什么。

In [ ]:
# extract index binary codes (represented as int)
arr = faiss.vector_to_array(index.codes)
arr

array([193, 189, 157, ..., 181, 245, 212], dtype=uint8)

In [ ]:
# we see that there are 1M of these values, 1 for each vector
arr.shape

(4000000,)

In [ ]:
# now translate them into the binary vector format
(((arr[:, None] & (1 << np.arange(nbits)))) > 0).astype(int)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

由此，我们可以可视化这 16 个存储桶中向量的分布 - 显示最常用的存储桶和一些空的存储桶。

![当nbits == 4时向量在不同桶中的分布。](https://cdn.sanity.io/images/vr8gru94/production/1fd2bc0b85bf0f8f8e4e1ac5f2121e85e3c788f7-1280x720.png)

当nbits == 4时向量在不同桶中的分布。

这个和我们之前的逻辑就是我们诊断上述分桶问题所需的全部。

------

## 在哪里使用 LSH

虽然 LSH 可以是一个 swift 索引，但它的准确性不如 Flat 索引。使用 Sift1M 数据集的越来越大的部分，使用768的nbits值实现了最佳召回分数（搜索次数过多时可能会获得更好的召回率）。

![回想一下索引向量的数量。 召回率以与详尽搜索结果的匹配百分比来衡量（使用 IndexFlatL2）。](https://cdn.sanity.io/images/vr8gru94/production/d9afc10469b3b9156d28a26c52c4892ae89281b6-1280x720.png)

回想一下索引向量的数量。召回率以与详尽搜索结果的匹配百分比来衡量（使用 IndexFlatL2）。

尽管值得注意的是，使用nbits值768仅返回比使用平面索引稍快的结果。

![搜索时间是不同索引大小和使用不同 nbits 值时 IndexFlatL2 搜索时间的一个因素。](https://cdn.sanity.io/images/vr8gru94/production/20049c873aebadb938af85b6d1b8c1d14e0cd0d4-1280x720.png)

搜索时间是不同索引大小和使用不同 nbits 值时 IndexFlatL2 搜索时间的一个因素。

更现实的召回率（同时保持合理的速度增长）接近 40%。

然而，不同的数据集大小和维度可能会产生巨大的差异。维数的增加意味着必须使用更高的nbits值来保持精度，但这仍然可以实现更快的搜索速度。这只是为每个用例和数据集找到适当平衡的一个例子。

当然，现在有很多用于向量相似性搜索的选项。扁平索引和 LSH 只是众多选项中的两个 - 选择正确的索引需要实验和专业知识的结合。

与往常一样，相似性搜索是不同索引和参数之间的平衡行为，以找到适合我们用例的最佳解决方案。

我们在本文中讨论了很多有关 LSH 的内容。希望这有助于消除人们对搜索领域最大的算法之一的任何困惑。

LSH 是一个复杂的主题，有[许多不同的方法](https://github.com/weedge/doraemon-nb/blob/main/faiss_lsh.ipynb)- 甚至在多个库中可以使用更多的实现。

除了 LSH 之外，我们还有更多适合高效相似性搜索的算法，例如[HNSW](https://www.pinecone.io/learn/series/faiss/hnsw/)、 IVF 和[PQ](https://www.pinecone.io/learn/series/faiss/product-quantization/)。您可以在我们的[向量搜索索引概述](https://www.pinecone.io/learn/vector-indexes/)中了解更多信息。

## References

- [Jupyter Notebooks](https://github.com/pinecone-io/examples/blob/master/learn/search/faiss-ebook/locality-sensitive-hashing-random-projection/)
- [1] [Google Searches](https://skai.io/monday-morning-metrics-daily-searches-on-google-and-other-google-facts/), Skai.io Blog

